![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/healthcare-nlp/08.1.Text_Classification_with_DocumentMLClassifier.ipynb)

# Colab Setup

In [ ]:
! pip install -q johnsnowlabs==5.1.0

In [ ]:
from google.colab import files
print('Please Upload your John Snow Labs License using the button below')
license_keys = files.upload()

In [ ]:
from johnsnowlabs import nlp, medical

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
nlp.install()

In [ ]:
from johnsnowlabs import nlp, medical, visual
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import pyspark.sql.functions as F

# Automatically load license data and start a session with all jars user has access to
spark = nlp.start(hardware_target="cpu")
spark

👌 Detected license file /content/spark_nlp_for_healthcare_spark_ocr_8283.json
👌 Launched cpu optimized session with with: 🚀Spark-NLP==5.1.0, 💊Spark-Healthcare==5.1.0, running on ⚡ PySpark==3.1.2


## 🔎 Models :


|index|model|
|-----:|:-----|
|1|[classifierml_ade](https://nlp.johnsnowlabs.com/2023/05/16/classifierml_ade_en.html)


## Load ADE Classification Dataset

In [ ]:
#downloading sample datasets
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/ADE_Corpus_V2/ADE-NEG.txt
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/ADE_Corpus_V2/DRUG-AE.rel

**ADE-Negative Dataset**

In [ ]:
df_neg= pd.read_csv("ADE-NEG.txt", header=None, delimiter="\t", names=["col1"])
df_neg['text'] =  df_neg.col1.str.split('NEG').str[1]
df_neg["category"] = "neg"
df_neg= df_neg[["text", "category"]]
df_neg.head()

,text,category
0,Clioquinol intoxication occurring in the trea...,neg
1,"""Retinoic acid syndrome"" was prevented with s...",neg
2,BACKGROUND: External beam radiation therapy o...,neg
3,"Although the enuresis ceased, she developed t...",neg
4,A 42-year-old woman had uneventful bilateral ...,neg


**ADE-Positive Dataset**

In [ ]:
df_pos= pd.read_csv("DRUG-AE.rel", header=None, delimiter="|")
df_pos["category"]= "pos"
df_pos.rename(columns={1: "text"}, inplace=True)
df_pos= df_pos[["text", "category"]]
df_pos.head()

,text,category
0,Intravenous azithromycin-induced ototoxicity.,pos
1,"Immobilization, while Paget's bone disease was...",pos
2,Unaccountable severe hypercalcemia in a patien...,pos
3,METHODS: We report two cases of pseudoporphyri...,pos
4,METHODS: We report two cases of pseudoporphyri...,pos


**Merging positive and negative datasets**

In [ ]:
ade_df= pd.concat([df_neg, df_pos]).sample(frac=1) #merge and shuffle the data
ade_df.head()

,text,category
3399,"Because of her severe symptoms, mifepristone ...",neg
2304,Various toxicities have been observed during ...,neg
8872,BACKGROUND: Reduction amniocentesis is used i...,neg
4382,The process is classified as a nonvasculitic ...,neg
8816,Total resection revealed that the lesions wer...,neg


In [ ]:
ade_df["category"].value_counts()

neg    16695
pos     6821
Name: category, dtype: int64

In [ ]:
ade_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23516 entries, 3399 to 8026
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      23516 non-null  object
 1   category  23516 non-null  object
dtypes: object(2)
memory usage: 551.2+ KB


In [ ]:
spark_df = spark.createDataFrame(ade_df)
spark_df.head(3)

[Row(text=' Because of her severe symptoms, mifepristone was administered for a period of 19 days.', category='neg'),
 Row(text=' Various toxicities have been observed during the treatment of advanced renal cell carcinoma with interferon-alpha (IFN-alpha) and/or interleukin-2 (IL-2).', category='neg'),
 Row(text=' BACKGROUND: Reduction amniocentesis is used in cases of severe polyhydramnios to decrease maternal discomfort and the risk of preterm labor.', category='neg')]

In [ ]:
spark_df.groupBy("category").count().show()
spark_df.printSchema()

+--------+-----+
|category|count|
+--------+-----+
|     pos| 6821|
|     neg|16695|
+--------+-----+

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)



**Get train & test set**

In [ ]:
train_data, test_data = spark_df.randomSplit([0.8, 0.2], seed = 100)

print("Train Dataset Count: " + str(train_data.count()))
print("Test Dataset Count: " + str(test_data.count()))

Train Dataset Count: 18793
Test Dataset Count: 4723


## DocumentMLClassifier with Linear SVM

The below pipeline uses the SVM classifier of `DocumentMLClassifier` with setting `setClassificationModelClass("svm")`.

In [ ]:
document = nlp.DocumentAssembler().setInputCol("text").setOutputCol("document")

token = nlp.Tokenizer().setInputCols("document").setOutputCol("token")

stemmer = nlp.Stemmer() \
    .setInputCols(["token"]) \
    .setOutputCol("stem")

classifier_svm = medical.DocumentMLClassifierApproach() \
    .setInputCols("stem") \
    .setLabelCol("category") \
    .setOutputCol("prediction") \
    .setClassificationModelClass("svm")

pipeline = nlp.Pipeline(stages=[document, token, stemmer, classifier_svm])

In [ ]:
svm_model = pipeline.fit(train_data)
result_svm = svm_model.transform(test_data).cache()
result_svm.show()

+--------------------+--------+--------------------+--------------------+--------------------+--------------------+
|                text|category|            document|               token|                stem|          prediction|
+--------------------+--------+--------------------+--------------------+--------------------+--------------------+
| Although gemcita...|     neg|[{document, 0, 24...|[{token, 1, 8, Al...|[{token, 1, 8, al...|[{category, 1, 24...|
| An inverse corre...|     neg|[{document, 0, 15...|[{token, 1, 2, An...|[{token, 1, 2, an...|[{category, 1, 15...|
| At delivery, the...|     neg|[{document, 0, 16...|[{token, 1, 2, At...|[{token, 1, 2, at...|[{category, 1, 16...|
| These difference...|     neg|[{document, 0, 27...|[{token, 1, 5, Th...|[{token, 1, 5, th...|[{category, 1, 27...|
|The possibility c...|     pos|[{document, 0, 20...|[{token, 0, 2, Th...|[{token, 0, 2, th...|[{category, 0, 20...|
|To our knowledge,...|     pos|[{document, 0, 17...|[{token, 0, 1, To...

In [ ]:
result_svm.printSchema()

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- token: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- va

In [ ]:
result_svm.select("text","prediction").show(4, truncate=100)

+----------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------+
|                                                                                                text|                                                                      prediction|
+----------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------+
| Although gemcitabine-based regimens have clear efficacy in refractory lymphoma, prior mediastina...|[{category, 1, 248, neg, {sentence -> 0, chunk -> 0, confidence -> 0.9864}, []}]|
| An inverse correlation was found between plasma potassium and doses of phosphate (plasma potassi...|[{category, 1, 153, neg, {sentence -> 0, chunk -> 0, confidence -> 0.8182}, []}]|
| At delivery, there were severe brachycephaly, hypoplasia of the anterior crani

In [ ]:
result_svm_df = result_svm.select('category','prediction.result').toPandas()
result_svm_df['result'] = result_svm_df.result.apply(lambda x : x[0])

print (classification_report(result_svm_df['category'], result_svm_df['result']))

              precision    recall  f1-score   support

         neg       0.89      0.97      0.93      3383
         pos       0.89      0.70      0.78      1340

    accuracy                           0.89      4723
   macro avg       0.89      0.83      0.85      4723
weighted avg       0.89      0.89      0.88      4723



## DocumentMLClassifier with Logistic Regression

In [ ]:
classifier_logreg = medical.DocumentMLClassifierApproach() \
    .setInputCols("token") \
    .setLabelCol("category") \
    .setOutputCol("prediction") \
    .setClassificationModelClass("logreg")\

pipeline = nlp.Pipeline(stages=[document, token, classifier_logreg])

logreg_model = pipeline.fit(train_data)
result_logreg = logreg_model.transform(test_data).cache()

result_logreg_df = result_logreg.select('category','prediction.result').toPandas()
result_logreg_df['result'] = result_logreg_df.result.apply(lambda x : x[0])

print (classification_report(result_logreg_df['category'], result_logreg_df['result']))

              precision    recall  f1-score   support

         neg       0.88      0.98      0.92      3383
         pos       0.91      0.65      0.76      1340

    accuracy                           0.88      4723
   macro avg       0.89      0.81      0.84      4723
weighted avg       0.89      0.88      0.88      4723



## Playing with Some Parameters

In [ ]:
classifier_logreg.extractParamMap()

{Param(parent='DocumentMLClassifierApproach_eff735e82285', name='lazyAnnotator', doc='Whether this AnnotatorModel acts as lazy in RecursivePipelines'): False,
 Param(parent='DocumentMLClassifierApproach_eff735e82285', name='labelCol', doc='column with the value result we are trying to predict.'): 'category',
 Param(parent='DocumentMLClassifierApproach_eff735e82285', name='maxIter', doc='maximum number of iterations.'): 10,
 Param(parent='DocumentMLClassifierApproach_eff735e82285', name='tol', doc='convergence tolerance after each iteration.'): 1e-06,
 Param(parent='DocumentMLClassifierApproach_eff735e82285', name='fitIntercept', doc='whether to fit an intercept term, default is true.'): True,
 Param(parent='DocumentMLClassifierApproach_eff735e82285', name='vectorizationModelPath', doc='specify the vectorization model if it has been already trained.'): '',
 Param(parent='DocumentMLClassifierApproach_eff735e82285', name='classificationModelPath', doc='specify the classification model if 

**Change `maxIter` parameter with `setMaxIter()`**

In [ ]:
classifier_logreg = medical.DocumentMLClassifierApproach() \
    .setInputCols("token") \
    .setLabelCol("category") \
    .setOutputCol("prediction") \
    .setClassificationModelClass("logreg")\
    .setMaxIter(5)

pipeline = nlp.Pipeline(stages=[document, token, classifier_logreg])

result_logreg = pipeline.fit(train_data).transform(test_data).cache()

result_logreg_df = result_logreg.select('category','prediction.result').toPandas()
result_logreg_df['result'] = result_logreg_df.result.apply(lambda x : x[0])

print (classification_report(result_logreg_df['category'], result_logreg_df['result']))

              precision    recall  f1-score   support

         neg       0.87      0.98      0.92      3383
         pos       0.93      0.63      0.75      1340

    accuracy                           0.88      4723
   macro avg       0.90      0.81      0.84      4723
weighted avg       0.89      0.88      0.88      4723



**Change `tol` (tolarance) parameter with `setTol()`**

In [ ]:
classifier_logreg = medical.DocumentMLClassifierApproach() \
    .setInputCols("token") \
    .setLabelCol("category") \
    .setOutputCol("prediction") \
    .setClassificationModelClass("logreg")\
    .setTol(1e-3)

pipeline = nlp.Pipeline(stages=[document, token, classifier_logreg])

result_logreg = pipeline.fit(train_data).transform(test_data).cache()

result_logreg_df = result_logreg.select('category','prediction.result').toPandas()
result_logreg_df['result'] = result_logreg_df.result.apply(lambda x : x[0])

print (classification_report(result_logreg_df['category'], result_logreg_df['result']))

              precision    recall  f1-score   support

         neg       0.88      0.98      0.92      3383
         pos       0.91      0.65      0.76      1340

    accuracy                           0.88      4723
   macro avg       0.89      0.81      0.84      4723
weighted avg       0.89      0.88      0.88      4723



**Change `maxTokenNgram` parameter with `setMaxTokenNgram()`**

In [ ]:
classifier_logreg = medical.DocumentMLClassifierApproach() \
    .setInputCols("token") \
    .setLabelCol("category") \
    .setOutputCol("prediction") \
    .setClassificationModelClass("logreg")\
    .setMaxTokenNgram(2)

pipeline = nlp.Pipeline(stages=[document, token, classifier_logreg])

result_logreg = pipeline.fit(train_data).transform(test_data).cache()

result_logreg_pf = result_logreg.select('category','prediction.result').toPandas()
result_logreg_pf['result'] = result_logreg_pf.result.apply(lambda x : x[0])

print (classification_report(result_logreg_pf['category'], result_logreg_pf['result']))

              precision    recall  f1-score   support

         neg       0.90      0.96      0.93      3383
         pos       0.88      0.72      0.79      1340

    accuracy                           0.89      4723
   macro avg       0.89      0.84      0.86      4723
weighted avg       0.89      0.89      0.89      4723



## Using More Annotators

In [ ]:
document = nlp.DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

token = nlp.Tokenizer() \
  .setInputCols(["document"]) \
  .setOutputCol("token")

normalizer = nlp.Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")

stopwords_cleaner = nlp.StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)

stemmer = nlp.Stemmer() \
    .setInputCols(["cleanTokens"]) \
    .setOutputCol("stem")

classifier_svm = medical.DocumentMLClassifierApproach() \
    .setInputCols("stem") \
    .setLabelCol("category") \
    .setOutputCol("prediction") \
    .setClassificationModelClass("svm")\
    .setMaxIter(5)

finisher = nlp.Finisher() \
    .setInputCols(["prediction"]) \
    .setOutputCols(["predictions"]) \
    .setCleanAnnotations(True)\



pipeline = nlp.Pipeline(stages=[document, token, normalizer, stopwords_cleaner, stemmer, classifier_svm, finisher])

model= pipeline.fit(train_data)

In [ ]:
sample_df = spark.createDataFrame([["We report two cases of pseudoporphyria caused by naproxen and oxaprozin."],
                                   ["Special attention should be paid when attempting to sample the endometrium in patients with mullerian abnormalities."]]).toDF("text")

result = model.transform(sample_df)
result.show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------+-----------+
|text                                                                                                                |predictions|
+--------------------------------------------------------------------------------------------------------------------+-----------+
|Special attention should be paid when attempting to sample the endometrium in patients with mullerian abnormalities.|[neg]      |
|We report two cases of pseudoporphyria caused by naproxen and oxaprozin.                                            |[pos]      |
+--------------------------------------------------------------------------------------------------------------------+-----------+

